In [66]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [67]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [68]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [69]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [70]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadSkillData/data/Media.pk2


In [71]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [72]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [73]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [74]:
_media_file_stream = open(media_pk2_path, 'rb')

In [75]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [76]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [77]:
blowfish_checksum_decoded = "Joymax Pak File"

In [78]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [79]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [80]:
Root = read_blocks_at(256)

In [81]:
blocks_in_memory = {
  "": Root
}

In [82]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [83]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [84]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### SkillMasteryData.txt

In [85]:
skill_mastery_data_file = os.path.join(text_data_directory, 'SkillMasteryData.txt')
skill_mastery_data_file

'server_dep/silkroad/textdata/SkillMasteryData.txt'

In [86]:
df_mastery_data= pd.DataFrame(columns=range(14))

lines = get_lines(skill_mastery_data_file)
for idx, line in enumerate(lines):
  values = line.split('\t')
  if len(values) < 14:
    continue
  df_mastery_data.loc[len(df_mastery_data)] = values
    


In [87]:
df_mastery_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,-1,캐릭터,xxx,0,xxx,xxx,255,2,0,0,0,xxx,xxx
1,2,-1,상인마스터리,xxx,0,xxx,xxx,255,2,0,0,0,xxx,xxx
2,3,-1,도적마스터리,xxx,0,xxx,xxx,255,2,0,0,0,xxx,xxx
3,4,-1,헌터마스터리,xxx,0,xxx,xxx,255,2,0,0,0,xxx,xxx
4,257,-1,비천검법101,UIIT_STT_MASTERY_VI,10,UIIT_STT_MASTERY_VI_EXPLANATION,UIIT_CTL_WEAPON_SKILL,0,0,2,3,0,icon\skillmastery\china\mastery_sword.ddj,icon\skillmastery\china\mastery_sword_focus.ddj
5,258,-1,흑살창법102,UIIT_STT_MASTERY_HEUK,9,UIIT_STT_MASTERY_HEUK_EXPLANATION,UIIT_CTL_WEAPON_SKILL,0,0,4,5,0,icon\skillmastery\china\mastery_spear.ddj,icon\skillmastery\china\mastery_spear_focus.ddj
6,259,-1,파천신궁103,UIIT_STT_MASTERY_PA,10,UIIT_STT_MASTERY_PA_EXPLANATION,UIIT_CTL_WEAPON_SKILL,0,0,6,0,0,icon\skillmastery\china\mastery_bow.ddj,icon\skillmastery\china\mastery_bow_focus.ddj
7,277,0,한빙면공111,UIIT_CTL_FORCE_SKILL,8,UIIT_STT_MASTERY_HAN_EXPLANATION,UIIT_CTL_FORCE_SKILL,1,1,0,0,0,icon\skillmastery\china\mastery_gigong.ddj,icon\skillmastery\china\mastery_gigong_focus.ddj
8,277,1,풍뢰비공112,UIIT_CTL_FORCE_SKILL,7,UIIT_STT_MASTERY_PUNG_EXPLANATION,UIIT_CTL_FORCE_SKILL,1,1,0,0,0,icon\skillmastery\china\mastery_gigong.ddj,icon\skillmastery\china\mastery_gigong_focus.ddj
9,277,2,화령신공113,UIIT_CTL_FORCE_SKILL,8,UIIT_STT_MASTERY_HWA_EXPLANATION,UIIT_CTL_FORCE_SKILL,1,1,0,0,0,icon\skillmastery\china\mastery_gigong.ddj,icon\skillmastery\china\mastery_gigong_focus.ddj
